In [1]:
import os
from os import walk

import pandas as pd
import numpy as np
import sys
import re

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from parse_data import *

%matplotlib inline

import math
import pickle

pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 1500)
pd.set_option('max_colwidth', 400)

In [2]:
main_folder_path = '..'

resultpath = main_folder_path + '/output_full_dataset/'

gold_standard_path = 'result_data/'

# Load Gold standard (## 1000Genomes)

## Load 2014 dataset

Loading and preprocessing of gold-standard dataset 

In [3]:
milleg_2014_df = pd.read_csv('../reference_data/1000G_hla_diversity_2014.txt', sep = " ", comment='#')

#Remove quotes
#Change name of Utah individuals from CEPH to CEU as seen in the 1000 genomes database:
#Remove samples with NaN (non typed alleles)
mille_gs_df = milleg_2014_df.replace({'\"':''}, regex=True).replace('CEPH','CEU').dropna()     

### Remove samples, which doesn't have exome data on 1000Genomes and create download links

In [4]:
import http.client
from urllib.parse import urlparse

def checkUrl(url):
    p = urlparse(url)
    conn = http.client.HTTPConnection(p.netloc)
    conn.request('HEAD', p.path)
    resp = conn.getresponse()
    return resp.status < 400

In [5]:
# #The results of this code are saved in two files, to save time

# #Check all exome links to create a list of id's with exome data
# gold_standard_id_list = list()

# #Save the valid urls for download later
# gold_standard_url_list = list()

# i = 0

# original_sample_list = list(mille_gs_df['id'])

# for identity in original_sample_list:
#     i += 1
#     if i % 100 == 0:
#         print(i, " iterations completed")
        
#     sbgroup = mille_gs_df[mille_gs_df['id'] == identity]['sbgroup'].iloc[0]         
#     wget_url = "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/data/{}/{}/exome_alignment/{}.alt_bwamem_GRCh38DH.20150826.{}.exome.cram".format(sbgroup, identity, identity, sbgroup, )
    
#     if checkUrl(wget_url):
#         gold_standard_id_list.append(identity)
#         gold_standard_url_list.append(wget_url)

# with open('../reference_data/gold_standard_url_list.txt', 'w') as outfile:
#     for entry in wget_url_list:
#         outfile.write(entry + '\n')
    
# with open('../reference_data/gold_standard_id_list.txt', 'w') as outfile:
#     for entry in valid_wes_id_list:
#         outfile.write(entry + '\n')

gold_standard_url_list = list()
with open('../reference_data/gold_standard_url_list.txt', 'r') as infile:
    for line in infile:
        gold_standard_url_list.append(line[:-1])   

gold_standard_id_list= list()
with open('../reference_data/gold_standard_id_list.txt', 'r') as infile:
    for line in infile:
        gold_standard_id_list.append(line[:-1])   

In [6]:
MG_exome_df = mille_gs_df[mille_gs_df['id'].isin(gold_standard_id_list)].drop('sbgroup', axis = 1)

#Replace 0000 with empty string:
MG_exome_df.replace('0000', '', inplace=True)

#MG_exome_df

### Double Typing
Merge rows, where a person has been typed twice. (this was only relevant for 2014 data)

In [7]:
#Check for non-identical rows
print(f"Number of unique ids: {len(set(list(MG_exome_df['id'])))}")
print(f"Number of total ids: {len(list(MG_exome_df['id']))}")

non_unique = list({x for x in list(MG_exome_df['id']) if list(MG_exome_df['id']).count(x) > 1})
non_unique_df = MG_exome_df[MG_exome_df['id'].isin(non_unique)]
non_unique_df

Number of unique ids: 819
Number of total ids: 822


,id,A,A.1,B,B.1,C,C.1,DRB1,DRB1.1,DQB1,DQB1.1
440,NA19119,23:01:01/23:07N/23:17/23:18/23:20,36:01,35:01:01/35:01:03/35:40N/35:42/35:57/35:94,49:01:01,04:01:01:01/04:01:01:02/04:01:01:03/04:09N/04:28/04:30/04:41,07:01:01/07:01:02/07:01:09/07:06/07:18/07:52,03:01:01:01/03:01:01:02,07:01:01:01/07:01:01:02,02:01:01/02:02/02:04,05:01:01
441,NA19119,23:01,36:01,35:01,49:01,04:01,07:01,,,,
480,NA19210,03:01,33:01,15:10,58:01,03:02:01/03:02:02/03:02:03,08:04,,,,
481,NA19210,03:01:01:01/03:01:01:02N/03:01:01:03/03:01:07/03:20/03:21N/03:26/03:37/03:45,33:01:01,15:10,58:01:01/58:11,03:02:01/03:02:02/03:02:03,08:04,03:01:01:01/03:01:01:02,13:01:01,03:01:01/03:01:04/03:09/03:19/03:21/03:22/03:24,06:03:01
486,NA19223,,,,,,,03:02,13:27,04:02,02:01
487,NA19223,30:01:01/30:01:02/30:24,33:03:01/33:03:03/33:15/33:25,41:04,42:01,17:01/17:02/17:03,17:01/17:02/17:03,03:02:01,13:27,02:01:01/02:02/02:04,04:02


In [8]:
# DQB1 and DQB1.1 are not sorted in order between for both entries of NA19223 - switch these around for one entry, so they match columnwise
non_unique_df.loc[486,'DQB1'] = '02:01'
non_unique_df.loc[486,'DQB1.1'] = '04:02'

non_unique_df

/tmp/ipykernel_1840502/2213804282.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_unique_df.loc[486,'DQB1'] = '02:01'
/tmp/ipykernel_1840502/2213804282.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_unique_df.loc[486,'DQB1.1'] = '04:02'


,id,A,A.1,B,B.1,C,C.1,DRB1,DRB1.1,DQB1,DQB1.1
440,NA19119,23:01:01/23:07N/23:17/23:18/23:20,36:01,35:01:01/35:01:03/35:40N/35:42/35:57/35:94,49:01:01,04:01:01:01/04:01:01:02/04:01:01:03/04:09N/04:28/04:30/04:41,07:01:01/07:01:02/07:01:09/07:06/07:18/07:52,03:01:01:01/03:01:01:02,07:01:01:01/07:01:01:02,02:01:01/02:02/02:04,05:01:01
441,NA19119,23:01,36:01,35:01,49:01,04:01,07:01,,,,
480,NA19210,03:01,33:01,15:10,58:01,03:02:01/03:02:02/03:02:03,08:04,,,,
481,NA19210,03:01:01:01/03:01:01:02N/03:01:01:03/03:01:07/03:20/03:21N/03:26/03:37/03:45,33:01:01,15:10,58:01:01/58:11,03:02:01/03:02:02/03:02:03,08:04,03:01:01:01/03:01:01:02,13:01:01,03:01:01/03:01:04/03:09/03:19/03:21/03:22/03:24,06:03:01
486,NA19223,,,,,,,03:02,13:27,02:01,04:02
487,NA19223,30:01:01/30:01:02/30:24,33:03:01/33:03:03/33:15/33:25,41:04,42:01,17:01/17:02/17:03,17:01/17:02/17:03,03:02:01,13:27,02:01:01/02:02/02:04,04:02


In [9]:
clean_duplicates_df = pd.DataFrame()
for column in ['A','A.1','B','B.1','C','C.1','DRB1','DRB1.1','DQB1','DQB1.1']:
    clean_duplicates_df[column] = non_unique_df.groupby(['id'])[column].apply('/'.join)
    
    #Remove potentaial starting '/'
    for identity in clean_duplicates_df.index:
        entry = clean_duplicates_df.loc[identity, column]
        
        if entry.startswith('/'):
            clean_duplicates_df.at[identity, column] = entry[1:]
        if entry.endswith('/'):
            clean_duplicates_df.at[identity, column] = entry[:-1]
        
clean_duplicates_df.reset_index(inplace=True)

In [10]:
clean_duplicates_df

,id,A,A.1,B,B.1,C,C.1,DRB1,DRB1.1,DQB1,DQB1.1
0,NA19119,23:01:01/23:07N/23:17/23:18/23:20/23:01,36:01/36:01,35:01:01/35:01:03/35:40N/35:42/35:57/35:94/35:01,49:01:01/49:01,04:01:01:01/04:01:01:02/04:01:01:03/04:09N/04:28/04:30/04:41/04:01,07:01:01/07:01:02/07:01:09/07:06/07:18/07:52/07:01,03:01:01:01/03:01:01:02,07:01:01:01/07:01:01:02,02:01:01/02:02/02:04,05:01:01
1,NA19210,03:01/03:01:01:01/03:01:01:02N/03:01:01:03/03:01:07/03:20/03:21N/03:26/03:37/03:45,33:01/33:01:01,15:10/15:10,58:01/58:01:01/58:11,03:02:01/03:02:02/03:02:03/03:02:01/03:02:02/03:02:03,08:04/08:04,03:01:01:01/03:01:01:02,13:01:01,03:01:01/03:01:04/03:09/03:19/03:21/03:22/03:24,06:03:01
2,NA19223,30:01:01/30:01:02/30:24,33:03:01/33:03:03/33:15/33:25,41:04,42:01,17:01/17:02/17:03,17:01/17:02/17:03,03:02/03:02:01,13:27/13:27,02:01/02:01:01/02:02/02:04,04:02/04:02


In [11]:
#Remove the duplicate rows from the full dataframe"
MG_exome_df = MG_exome_df[~MG_exome_df['id'].isin(non_unique)]

#Add back the clean duplicate rows:
MG_exome_df = MG_exome_df.append(clean_duplicates_df, sort=False)

#Reset index
MG_exome_df.reset_index(inplace=True, drop = True)

#Check that only uniwue entries exist now.
print(len(set(list(MG_exome_df['id']))))
print(len(list(MG_exome_df['id'])))

819
819


/tmp/ipykernel_1840502/429365880.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  MG_exome_df = MG_exome_df.append(clean_duplicates_df, sort=False)


In [12]:
#Set id as index
MG_exome_df.set_index('id', inplace=True)

## Separate multiple predictions to interable lists and mark untyped alleles:

In [13]:
#Remember entries, which are not typed
non_typed_samples = list()

#Furthermore - in cases, where 2 field resolution is not available, use 2018 data.
for identity in list(MG_exome_df.index):
    
    for col in MG_exome_df.columns:
        #get list of predictions
        predictions_list = MG_exome_df.loc[identity,col].split('/')
        
        #Put gene name in front of all entries in the list
        for i in range(len(predictions_list)):
            predictions_list[i] = col.split('.')[0] + "*" + predictions_list[i]
        
        #Check, that all entries in the list have at least four field resolution:
        for pred in predictions_list:
            
            #Remove all non valid entries
            if convert_to_two_field(pred) == None:
                predictions_list.remove(pred)
        
        #Note, if no proper typing was made in 2014
        if len(predictions_list) == 0:
            non_typed_samples.append(identity)
            predictions_list = ['not_typed_in_2014']
            
        #Convert to P-type resolution:
        
 
        MG_exome_df.at[identity,col] = predictions_list

print("Non typed samples in 2014:")
non_typed_samples

Non typed samples in 2014:


['NA12234', 'NA12249']

In [14]:
#Save 2014 clean dataset:

MG_exome_df.to_pickle('../reference_data/1000G_2014_cleaned.pkl')

## Merge Alleles in original dataframe

In [15]:
#Merge haplotypes
MG_exome_df['A_merged']= MG_exome_df[['A', 'A.1']].apply(lambda x: list(x), axis=1)
MG_exome_df['B_merged']= MG_exome_df[['B', 'B.1']].apply(lambda x: list(x), axis=1)
MG_exome_df['C_merged']= MG_exome_df[['C', 'C.1']].apply(lambda x: list(x), axis=1)
MG_exome_df['DRB1_merged']= MG_exome_df[['DRB1', 'DRB1.1']].apply(lambda x: list(x), axis=1)
MG_exome_df['DQB1_merged']= MG_exome_df[['DQB1', 'DQB1.1']].apply(lambda x: list(x), axis=1)

MG_exome_merged_df = MG_exome_df.drop(columns=['A', 'A.1', 'B', 'B.1', 'C', 'C.1', 'DRB1', 'DRB1.1','DQB1', 'DQB1.1'])

MG_exome_merged_df.rename(columns={"A_merged": "A", "B_merged": "B", "C_merged": "C", "DRB1_merged": "DRB1", "DQB1_merged": "DQB1"}, inplace = True)

## Load and convert 2018 data for error correction in 2014 dataset

This data is not used itself directly, but fills out the gaps in the 2014 dataset loaded later

In [16]:
#10 samples analysed in ATHLATES and with results taken from 2018 dataset.
ATHLATES_samples = ["HG01756", "HG01757", "HG01872", "HG01873", "HG01886", "HG01953", "HG01968", "HG02014", "HG02057", "NA20313"]

In [17]:
gs_2018_filepath = '../reference_data/2018_1129_HLA_types_full_1000_Genomes_Project_panel.txt'

gs_2018_raw_df = pd.read_csv(gs_2018_filepath, sep = "\t", comment="#")

#Rename columns
gs_2018_raw_df.rename(columns={'Sample ID': 'id', 'Population': 'sbgroup', 'HLA-A 1': 'A', 'HLA-A 2': 'A.1', 'HLA-B 1': 'B',
                            'HLA-B 2': 'B.1', 'HLA-C 1': 'C', 'HLA-C 2': 'C.1', 'HLA-DQB1 1': 'DQB1', 'HLA-DQB1 2': 'DQB1.1',
                            'HLA-DRB1 1': 'DRB1' , 'HLA-DRB1 2': 'DRB1.1' }, inplace=True)

#Set id as index
gs_2018_raw_df.set_index('id', inplace=True)

#Remove samples with NaN (non typed alleles)
#gs_2018_df = gs_2018_raw_df.dropna()  
gs_2018_df = gs_2018_raw_df        

#Find entries with an *, indicating a former mistake in the 2014 dataset
changed_sample_indexes = list()

for index in gs_2018_df.index:
    for col in gs_2018_df.columns[1:]:
        if isinstance(gs_2018_df.loc[index][col], float):
            if math.isnan(gs_2018_df.loc[index][col]):
                continue
        for entry in gs_2018_df.loc[index][col]:
            if '*' in entry:              
                changed_sample_indexes.append(index)

#Include samples, which are missing as well.
changed_sample_indexes += non_typed_samples

#Include the 10 high coverage samples:
changed_sample_indexes += ATHLATES_samples

#make relevant dataframe smaller
corrections_2018_df_raw = gs_2018_df.loc[set(changed_sample_indexes)]

#Limit the 2018 dataframe to 2014 samples and the 10 high coverage samples 
corrections_2018_df = corrections_2018_df_raw[corrections_2018_df_raw.index.isin(list(MG_exome_df.index) + ATHLATES_samples)]

#Make dataframe for writing into 2014 dataframe:

#Remove * from entry:
corrections_2018_df_no_asterix = corrections_2018_df.copy()
corrections_2018_df_no_asterix=corrections_2018_df_no_asterix.replace({'\*':''}, regex=True)

#Add allele name in front of the resolution:
for identity in corrections_2018_df_no_asterix.index:
    for col in corrections_2018_df_no_asterix.columns[2:]:
        entry = corrections_2018_df_no_asterix.loc[identity][col]
        if isinstance(entry, float):
            if math.isnan(entry):
                continue
        entrylist = corrections_2018_df_no_asterix.loc[identity][col].split('/')       
        
        entrylist = list(set([col.split('.')[0] + "*" + i.split('*')[0] for i in entrylist]))
        
        corrections_2018_df_no_asterix.at[identity,col] = entrylist

len(corrections_2018_df)

/tmp/ipykernel_1840502/1275223798.py:36: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  corrections_2018_df_raw = gs_2018_df.loc[set(changed_sample_indexes)]


74

In [18]:
#Merge alleles into one column:

corrections_2018_df_no_asterix['A_merged']= corrections_2018_df_no_asterix[['A', 'A.1']].apply(lambda x: list(x), axis=1)
corrections_2018_df_no_asterix['B_merged']= corrections_2018_df_no_asterix[['B', 'B.1']].apply(lambda x: list(x), axis=1)
corrections_2018_df_no_asterix['C_merged']= corrections_2018_df_no_asterix[['C', 'C.1']].apply(lambda x: list(x), axis=1)
corrections_2018_df_no_asterix['DRB1_merged']= corrections_2018_df_no_asterix[['DRB1', 'DRB1.1']].apply(lambda x: list(x), axis=1)
corrections_2018_df_no_asterix['DQB1_merged']= corrections_2018_df_no_asterix[['DQB1', 'DQB1.1']].apply(lambda x: list(x), axis=1)

corrections_2018_merged_df = corrections_2018_df_no_asterix.drop(columns=['A', 'A.1', 'B', 'B.1', 'C', 'C.1', 'DRB1', 'DRB1.1','DQB1', 'DQB1.1'])

corrections_2018_merged_df = corrections_2018_merged_df.rename(columns={"A_merged": "A", "B_merged": "B", "C_merged": "C", "DRB1_merged": "DRB1", "DQB1_merged": "DQB1"})
corrections_2018_merged_df.head()

,Region,sbgroup,A,B,C,DRB1,DQB1
id,,,,,,,
NA11994,EUR,CEU,"[[A*01:01], [A*11:01]]","[[B*07:02], [B*51:01]]","[[C*07:02], [C*15:13]]","[[DRB1*04:02], [DRB1*04:04]]","[[DQB1*03:02], [DQB1*03:02]]"
NA06986,EUR,CEU,"[[A*03:01], [A*32:01]]","[[B*44:03], [B*44:03]]","[[C*04:01], [C*16:01]]","[[DRB1*07:01], [DRB1*07:01]]","[[DQB1*02:02], [DQB1*02:02]]"
NA19116,AFR,YRI,"[[A*23:01], [A*33:03]]","[[B*07:02], [B*39:24]]","[[C*07:01], [C*07:02]]","[[DRB1*03:02], [DRB1*13:02]]","[[DQB1*04:02], [DQB1*06:09]]"
NA12156,EUR,CEU,"[[A*01:01], [A*11:01]]","[[B*50:01], [B*51:01]]","[[C*06:06], [C*15:02]]","[[DRB1*04:07], [DRB1*07:01]]","[[DQB1*02:02], [DQB1*03:01]]"
HG01356,AMR,CLM,"[[A*24:02], [A*68:01]]","[[B*35:43], [B*18:01]]","[[C*01:02], [C*05:01]]","[[DRB1*03:01], [DRB1*04:92]]","[[DQB1*02:01], [DQB1*03:02]]"


In [19]:
#corrections_2018_merged_df[[len(str(corrections_2018_merged_df.loc[i]['DQB1'])) < 11 for i in corrections_2018_merged_df.index]]

In [20]:
#Update samples, where typing was not performed in 2014:

for identity in non_typed_samples:
    for col in list(MG_exome_merged_df.columns)[1:]:
        entry = MG_exome_merged_df.loc[identity,col]
        if ['not_typed_in_2014'] in entry:
            
            column = col.split('.')[0]
                
            new_type = corrections_2018_merged_df.loc[identity][column]

            old_type = MG_exome_merged_df.loc[identity,column]

            corrected_typing = list()

            for new_allele in list(new_type):
                if (new_allele in list(old_type)[0]):
                    corrected_typing.append(list(old_type)[0][0])
                elif new_allele in list(old_type)[1]:
                    corrected_typing.append(list(old_type)[0][1])
                else:
                    corrected_typing.append(new_allele)

            MG_exome_merged_df.at[identity,column] = corrected_typing
            
            

In [21]:
#Perform update of samples, where mistakes have been found

changed_sample_indexes = list()

#Log changed samples
changed_typing_dict = dict()

#Loop over the samples, which needs to be corrected
for identity in list(corrections_2018_df.index):
    
    #Loop over relevant columns (genes)
    for col in corrections_2018_df.columns[3:]:
        
        #Loop over alleles
        for entry in corrections_2018_df[corrections_2018_df.index == identity][col]:
            
            if isinstance(entry, float):
                if math.isnan(entry):
                    continue
            #If allele is corrected since 2014 - update it
            if '*' in entry:
                column = col.split('.')[0]
                
                new_type = corrections_2018_merged_df.loc[identity][column]
                
                old_type = MG_exome_merged_df.loc[identity,column]

                if identity not in changed_typing_dict:
                    changed_typing_dict[identity] = dict()
                    changed_typing_dict[identity][col] = (old_type, new_type)
                else:
                    changed_typing_dict[identity][col] = (old_type, new_type)
                
                corrected_typing = list()
                
                for new_allele in list(new_type):
                    if (new_allele in list(old_type)[0]):
                        corrected_typing.append(list(old_type)[0][0])
                    elif new_allele in list(old_type)[1]:
                        corrected_typing.append(list(old_type)[0][1])
                    else:
                        corrected_typing.append(new_allele)
                
                MG_exome_merged_df.at[identity,column] = corrected_typing
            
    #Add the 10 high coverage samples to MG_exome_merged
    if identity in ATHLATES_samples:
        
        high_cov_prediction_row = corrections_2018_merged_df.loc[identity][['A','B','C','DRB1','DQB1']]        
        MG_exome_merged_df = MG_exome_merged_df.append(high_cov_prediction_row)    

/tmp/ipykernel_1840502/714003344.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  MG_exome_merged_df = MG_exome_merged_df.append(high_cov_prediction_row)
/tmp/ipykernel_1840502/714003344.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  MG_exome_merged_df = MG_exome_merged_df.append(high_cov_prediction_row)
/tmp/ipykernel_1840502/714003344.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  MG_exome_merged_df = MG_exome_merged_df.append(high_cov_prediction_row)
/tmp/ipykernel_1840502/714003344.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  MG_exome_merged_df = MG_exome_merged_df.append(high_cov_prediction_row)
/tmp/ipykern

In [22]:
#Note samples, where the update affects the allele typing in P group resolution
num_updated_loci = 0
num_identical_p_group = 0
num_differing_p_group = 0

for sample, locus_dict in changed_typing_dict.items():
    for locus, updated_typing in locus_dict.items():
        num_updated_loci += 1
        (old_type, new_type) = updated_typing
        for i in range(2):
            overlap_p_group = False
            for old_type_allele in sorted(old_type)[i]:
                if convert_to_p_group(old_type_allele) == convert_to_p_group(sorted(new_type)[i][0]):
                    overlap_p_group = True
            
            num_identical_p_group += int(overlap_p_group == True)
            num_differing_p_group += int(overlap_p_group == False)
            if overlap_p_group == False:
                print(sample)
                # print(old_type)
                # print(new_type)
                # print(convert_to_p_group(old_type_allele))
                # print(convert_to_p_group(new_type[i][0]))
                # print()

            

NA19116
NA19116
NA12156
NA19206
HG01389
NA12005
NA12287
NA11840
NA19129
NA19129
NA19239
NA10847
NA11832
HG01124
NA19159
NA18507
NA18504
NA19138
NA19138
NA19138
NA19093
NA12058
NA12058
NA19171
NA19319
NA19141


In [23]:
num_identical_p_group

128

In [24]:
num_differing_p_group

26

In [25]:
num_updated_loci

77

## Make URL list for 10 high coverage WES samples (Validated in ATHLATES)

In [26]:
print(ATHLATES_samples) 

high_cov_url_list = list()

# for identity in ATHLATES_samples:
       
#     sbgroup = m2018_gs_df.loc[identity]['sbgroup']
#     wget_url = "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/data/{}/{}/exome_alignment/{}.alt_bwamem_GRCh38DH.20150826.{}.exome.cram".format(sbgroup, identity, identity, sbgroup, )
    
#     if checkUrl(wget_url):
#         high_cov_url_list.append(wget_url)

# with open('../reference_data/high_cov_url_list.txt', 'w') as outfile:
#     for entry in high_cov_url_list:
#         outfile.write(entry + '\n')
    

high_cov_url_list = list()
with open('../reference_data/high_cov_url_list.txt', 'r') as infile:
    for line in infile:
        high_cov_url_list.append(line[:-1])     

['HG01756', 'HG01757', 'HG01872', 'HG01873', 'HG01886', 'HG01953', 'HG01968', 'HG02014', 'HG02057', 'NA20313']


## Write config.yaml file for Snakemake workflow (both 2014 data and high cov data)

In [27]:
full_sample_urllist = list(set(gold_standard_url_list)) + high_cov_url_list
full_sample_id_list = list(set(gold_standard_id_list)) + ATHLATES_samples

configfile_1 = '../Snakemake/config_all.yaml'

with open(configfile_1, 'w') as outfile:
    outfile.write('sample_urlist:\n')
    for entry in full_sample_urllist:
        entry_name = entry.split('/')[9]
        outfile.write("  " + entry_name + ": " +  "\"" + entry + "\"" + '\n')

In [28]:
MG_exome_merged_df.head()

,A,B,C,DRB1,DQB1
id,,,,,
NA06985,"[[A*03:01], [A*02:01]]","[[B*07:02:01], [B*57:01]]","[[C*07:02], [C*06:02:01:01]]","[[DRB1*15:01], [DRB1*15:01]]","[[DQB1*06:02:01], [DQB1*06:02:01]]"
NA06986,"[[A*03:01], [A*32:01:01, A*32:01:02]]","[[B*44:03:01, B*44:03:03, B*44:03:04], [B*44:03:01, B*44:03:03, B*44:03:04]]","[[C*04:01], [C*16:01]]","[[DRB1*07:01], [DRB1*07:01]]","[[DQB1*02:02], [DQB1*02:02]]"
NA06994,"[[A*02:01], [A*32:01:01, A*32:01:02]]","[[B*40:02], [B*08:01]]","[[C*02:02:02], [C*07:01, C*07:06]]","[[DRB1*01:01], [DRB1*04:04]]","[[DQB1*05:01], [DQB1*03:02]]"
NA07000,"[[A*02:01], [A*68:01:02]]","[[B*44:02], [B*40:01]]","[[C*03:03, C*03:04], [C*07:04]]","[[DRB1*03:01], [DRB1*11:01:01, DRB1*11:01:08]]","[[DQB1*02:01], [DQB1*03:01]]"
NA07037,"[[A*30:01], [A*31:01]]","[[B*15:10], [B*40:01]]","[[C*03:04:02], [C*03:04]]","[[DRB1*04:04], [DRB1*13:02]]","[[DQB1*03:02], [DQB1*06:04]]"


#

# Convert all predictions to 2-field resolution

In [29]:
gs_two_field_df = MG_exome_merged_df.copy()

#Loop over all entries and update the three gold standard dataframes, so they fit the idividual resolution
for identity in list(MG_exome_merged_df.index):
    for gene in list(MG_exome_merged_df.columns):
        
        old_pred_list = MG_exome_merged_df.loc[identity,gene]
        
        gene_pred_two_field = list()
    
        #Loop over the two alleles
        for allele_list in old_pred_list:             
            allele_pred_two_field = list()

            #Convert each prediction to it's respective correct format:
            for allele in allele_list:
                allele_pred_two_field.append(convert_to_two_field(allele))
                
            #Merge the two alleles for each gene in a list
            gene_pred_two_field.append(list(set(allele_pred_two_field)))

        #Update dataframes wit0h the new predictions
        gs_two_field_df.at[identity,gene] = gene_pred_two_field

# Save Dataframes as pickle objects

In [30]:
MG_exome_merged_df.to_pickle(gold_standard_path + "MG_exome_merged_df.pkl")

gs_two_field_df.to_pickle(gold_standard_path + "gs_two_field_df.pkl")

In [31]:
gs_two_field_df[gs_two_field_df.index.isin(ATHLATES_samples)]

,A,B,C,DRB1,DQB1
id,,,,,
HG01872,"[[A*11:02], [A*24:07]]","[[B*27:04], [B*39:05]]","[[C*08:01], [C*12:02]]","[[DRB1*08:03], [DRB1*12:02]]","[[DQB1*03:01], [DQB1*06:01]]"
HG01757,"[[A*01:01], [A*02:01]]","[[B*18:01], [B*57:01]]","[[C*07:01], [C*07:01]]","[[DRB1*03:01], [DRB1*07:01]]","[[DQB1*02:01], [DQB1*03:03]]"
HG01756,"[[A*66:01], [A*30:02]]","[[B*18:01], [B*41:02]]","[[C*05:01], [C*17:03]]","[[DRB1*03:01], [DRB1*03:01]]","[[DQB1*02:01], [DQB1*02:01]]"
HG01953,"[[A*02:01], [A*02:11]]","[[B*15:04], [B*35:05]]","[[C*01:02], [C*04:01]]","[[DRB1*04:11], [DRB1*09:01]]","[[DQB1*03:02], [DQB1*03:03]]"
HG01873,"[[A*02:03], [A*03:01]]","[[B*35:03], [B*55:02]]","[[C*04:01], [C*12:03]]","[[DRB1*08:02], [DRB1*14:05]]","[[DQB1*04:02], [DQB1*05:03]]"
NA20313,"[[A*03:01], [A*68:02]]","[[B*35:01], [B*53:01]]","[[C*04:01], [C*04:01]]","[[DRB1*08:04], [DRB1*04:05]]","[[DQB1*03:01], [DQB1*03:02]]"
HG01968,"[[A*02:01], [A*68:01]]","[[B*07:02], [B*40:02]]","[[C*03:04], [C*07:02]]","[[DRB1*01:03], [DRB1*09:01]]","[[DQB1*03:03], [DQB1*05:01]]"
HG02057,"[[A*02:03], [A*31:01]]","[[B*48:01], [B*13:01]]","[[C*03:03], [C*03:04]]","[[DRB1*11:01], [DRB1*13:12]]","[[DQB1*03:01], [DQB1*03:01]]"
HG02014,"[[A*02:01], [A*36:01]]","[[B*35:01], [B*40:01]]","[[C*03:04], [C*04:01]]","[[DRB1*01:01], [DRB1*15:01]]","[[DQB1*05:01], [DQB1*06:02]]"


# NOTES

In [32]:
# gs_e_group_df = MG_exome_merged_df.copy()

# #Loop over all entries and update the three gold standard dataframes, so they fit the idividual resolution
# for identity in list(MG_exome_merged_df.index):
#     for gene in list(MG_exome_merged_df.columns):
        
#         old_pred_list = MG_exome_merged_df.loc[identity,gene]
        
#         gene_pred_e_group = list()
    
    
#         #Loop over the two alleles
#         for allele_list in old_pred_list:
            
#             allele_pred_e_group = list()
            
#             #Convert each prediction to it's respective correct format:
#             for allele in allele_list:
#                 allele_pred_e_group.append(convert_to_e_group(allele))
                                
        
#             #Merge the two alleles for each gene in a list
#             gene_pred_e_group.append(list(set(allele_pred_e_group)))

#         #Update dataframes wit0h the new predictions
#         gs_e_group_df.at[identity,gene] = gene_pred_e_group